# Compare MendelImpute against Minimac4 and Beagle5 on simulated data

In compare 2, we distinguish typed and untyped SNPs. 10% of typed SNPs are missing and only 50% of all SNPs are typed. 

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using SparseArrays
using JLD2, FileIO, JLSO
using ProgressMeter
using GroupSlices
using ThreadPools
using LinearAlgebra
# using Plots
# using ProfileView

In [7]:
# impute only (8 thread)
Random.seed!(2020)
d       = 1000
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.maxd$d.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph2 = phase(tgtfile, reffile, outfile = outfile, max_d = d,
    phase = true);

# import imputed result and compare with true
X_mendel2 = convert_gt(Float32, outfile, trans=true)
# X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz", trans=true)
n, p = size(X_mendel2)
println("error_rate = ", sum(X_mendel2 .!= X_complete) / n / p)
# rm(outfile, force=true)

Number of threads = 8
Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07


Total windows = 64, averaging ~ 692 unique haplotypes per window.

Timings: 
    Data import                     = 8.87686 seconds
        import target data             = 8.15481 seconds
        import compressed haplotypes   = 0.722049 seconds
    Computing haplotype pair        = 2.27887 seconds
        BLAS3 mul! to get M and N      = 0.0642376 seconds per thread
        haplopair search               = 1.84433 seconds per thread
        initializing missing           = 0.165215 seconds per thread
        allocating and viewing         = 0.0264083 seconds per thread
        index conversion               = 0.000448376 seconds per thread
    Phasing by win-win intersection = 0.0374169 seconds
        Window-by-window intersection  = 0.0263847 seconds per thread
        Breakpoint search              = 0.00502967 seconds per thread
        Recording result               = 0.00247853 seconds per thread
    Imputation                     = 1.06556 seconds
        Imputing missing      

# Simulate data

### Step 0. Install `msprime`

[msprime download Link](https://msprime.readthedocs.io/en/stable/installation.html).

Some people might need to activate conda environment via `conda config --set auto_activate_base True`. You can turn it off once simulation is done by executing `conda config --set auto_activate_base False`.


### Step 1. Simulate data in terminal

```
python3 msprime_script.py 40000 10000 10000000 2e-8 2e-8 2020 > full.vcf
```

Arguments: 
+ Number of haplotypes = 40000
+ Effective population size = 10000 ([source](https://www.the-scientist.com/the-nutshell/ancient-humans-more-diverse-43556))
+ Sequence length = 10 million (same as Beagle 5's choice)
+ Rrecombination rate = 2e-8 (default)
+ mutation rate = 2e-8 (default)
+ seed = 2019

### Step 2: Convert simulated haplotypes to reference haplotypes and target genotype files

In [13]:
cd("./compare2/")
function filter_and_mask(maf)
    # filter chromosome data for unique snps
    println("filtering for unique snps")
    data = "full.vcf"
    full_record_index = .!find_duplicate_marker(data)
    @time VCFTools.filter(data, full_record_index, 1:nsamples(data), 
        des = "full.uniqueSNPs.vcf.gz")

    # summarize data
    println("summarizing data")
    total_snps, samples, _, _, _, maf_by_record, _ = gtstats("full.uniqueSNPs.vcf.gz")

    # generate target panel with all snps
    println("generating complete target panel")
    n = 1000
    sample_idx = falses(samples)
    sample_idx[1:n] .= true
    shuffle!(sample_idx)
    @time VCFTools.filter("full.uniqueSNPs.vcf.gz", 1:total_snps, 
        sample_idx, des = "target.full.vcf.gz", allow_multiallelic=false)

    also generate reference panel without target samples
    println("generating reference panel without target samples")
    @time VCFTools.filter("full.uniqueSNPs.vcf.gz", 1:total_snps, 
        .!sample_idx, des = "ref.excludeTarget.vcf.gz", allow_multiallelic=false)

    # generate target file with 1000 samples and typed snps with certain maf
    println("generating target file with typed snps only")
    my_maf = findall(x -> x > maf, maf_by_record)  
    p = length(my_maf)
    record_idx = falses(total_snps)
    record_idx[my_maf] .= true
    @time VCFTools.filter("full.uniqueSNPs.vcf.gz", record_idx, sample_idx, 
        des = "target.typedOnly.maf$maf.vcf.gz", allow_multiallelic=false)

    # unphase and mask 1% entries in target file
    println("unphasing and masking entries in target file with typed snps only")
    masks = falses(p, n)
    missingprop = 0.1
    for j in 1:n, i in 1:p
        rand() < missingprop && (masks[i, j] = true)
    end
    @time mask_gt("target.typedOnly.maf$maf.vcf.gz", masks, 
        des="target.typedOnly.maf$maf.masked.vcf.gz", unphase=true)

    # finally compress reference file to jlso format
    widths  = [32, 64, 128, 256, 512]
    reffile = "ref.excludeTarget.vcf.gz"
    tgtfile = "target.typedOnly.maf$maf.masked.vcf.gz"
    H, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt = convert_ht(Bool, reffile, trans=true, save_snp_info=true, msg="importing reference data...")
    X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...")
    for width in widths
        outfile = "ref.excludeTarget.w$width.jlso"
        @time compress_haplotypes(H, X, outfile, X_pos, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt, width)
    end
end
Random.seed!(2020)
maf = 0.01
@time filter_and_mask(maf)

summarizing data


Progress: 100%|█████████████████████████████████████████| Time: 0:08:31


generating complete target panel
generating target file with typed snps only
193.850375 seconds (3.78 G allocations: 288.466 GiB, 15.40% gc time)
unphasing and masking entries in target file with typed snps only
  8.499865 seconds (73.79 M allocations: 5.496 GiB, 7.70% gc time)


importing reference data...100%|████████████████████████| Time: 0:03:23
Importing genotype file...100%|█████████████████████████| Time: 0:00:06


137.175658 seconds (10.71 M allocations: 16.615 GiB, 1.54% gc time)
 95.677300 seconds (2.16 M allocations: 9.058 GiB, 1.69% gc time)
 76.646906 seconds (1.95 M allocations: 5.037 GiB, 1.10% gc time)
 65.061100 seconds (1.72 M allocations: 2.962 GiB, 0.32% gc time)
 58.832663 seconds (1.49 M allocations: 1.891 GiB, 0.42% gc time)
1377.550815 seconds (12.76 G allocations: 1.129 TiB, 8.42% gc time)


In [2]:
# compress reference file to jlso format
max_d   = [1000]
reffile = "./compare2/ref.excludeTarget.vcf.gz"
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
H, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt = convert_ht(Bool, reffile, trans=true, save_snp_info=true, msg="importing reference data...")
X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...")
for d in max_d
    outfile = "./compare2/ref.excludeTarget.maxd$d.jlso"
    @time compress_haplotypes(H, X, outfile, X_pos, H_sampleID, H_chr, H_pos, H_ids, 
        H_ref, H_alt, d)
end

importing reference data...100%|████████████████████████| Time: 0:03:26
Importing genotype file...100%|█████████████████████████| Time: 0:00:06


161.157003 seconds (18.25 M allocations: 2.493 GiB, 0.25% gc time)


In [34]:
# load jlso
@time loaded = JLSO.load("./compare2/ref.excludeTarget.maxd1000.jlso")
compressed_Hunique = loaded[:compressed_Hunique];

  0.830604 seconds (1.76 M allocations: 154.475 MiB)


In [32]:
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.vcf.gz"
@show nrecords(tgtfile), nsamples(tgtfile)
@show nrecords(reffile), nsamples(reffile);

(nrecords(tgtfile), nsamples(tgtfile)) = (36874, 1000)
(nrecords(reffile), nsamples(reffile)) = (89913, 19000)


# MendelImpute with dynamic programming

In [22]:
# keep best pair only (1 thread)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:11
Merging breakpoints...100%|█████████████████████████████| Time: 0:01:12
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.07908 seconds
    Computing haplotype pair        = 11.4215 seconds
        BLAS3 mul! to get M and N      = 0.209483 seconds per thread
        haplopair search               = 9.27352 seconds per thread
        supplying constant terms       = 0.0368894 seconds per thread
        finding redundant happairs     = 0.800724 seconds per thread
    Phasing by dynamic programming  = 72.6589 seconds
    Imputation                      = 7.44617 seconds

 99.605137 seconds (77.18 M allocations: 7.509 GiB, 1.09% gc time)
error_rate = 0.0003560219323123464


In [3]:
# keep best pair only (8 threads)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:11
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.06139 seconds
    Computing haplotype pair        = 3.04445 seconds
        BLAS3 mul! to get M and N      = 0.0601239 seconds per thread
        haplopair search               = 2.45815 seconds per thread
        supplying constant terms       = 0.00534291 seconds per thread
        finding redundant happairs     = 0.155477 seconds per thread
    Phasing by dynamic programming  = 11.9017 seconds
    Imputation                      = 7.78007 seconds

 30.787638 seconds (77.18 M allocations: 7.555 GiB, 3.16% gc time)
error_rate = 0.0003560219323123464


# MendelImpute with intersecting haplotype sets

In [5]:
# impute only (8 thread)
Random.seed!(2020)
d       = 1000
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.maxd$d.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:09
Writing to file...100%|█████████████████████████████████| Time: 0:00:08


Total windows = 64, averaging ~ 692 unique haplotypes per window.

Timings: 
    Data import                     = 11.2971 seconds
    Computing haplotype pair        = 3.34432 seconds
        BLAS3 mul! to get M and N      = 0.102085 seconds per thread
        haplopair search               = 2.82811 seconds per thread
        initializing missing           = 0.211092 seconds per thread
        allocating and viewing         = 0.0287914 seconds per thread
        index conversion               = 0.000847365 seconds per thread
    Phasing by win-win intersection = 0.0654008 seconds
        Window-by-window intersection  = 0.0452051 seconds per thread
        Breakpoint search              = 0.00805598 seconds per thread
        Recording result               = 0.00531286 seconds per thread
    Imputation                      = 9.21823 seconds

 23.926207 seconds (77.16 M allocations: 6.401 GiB, 4.77% gc time)
error_rate = 7.199181430938797e-5


In [7]:
# phase (8 thread)
Random.seed!(2020)
d       = 1000
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.maxd$d.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false, phase=true);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:08
Writing to file...100%|█████████████████████████████████| Time: 0:00:08


Total windows = 64, averaging ~ 692 unique haplotypes per window.

Timings: 
    Data import                     = 10.3269 seconds
    Computing haplotype pair        = 2.79167 seconds
        BLAS3 mul! to get M and N      = 0.0938853 seconds per thread
        haplopair search               = 2.33387 seconds per thread
        initializing missing           = 0.1796 seconds per thread
        allocating and viewing         = 0.0255029 seconds per thread
        index conversion               = 0.0006521 seconds per thread
    Phasing by win-win intersection = 0.0448282 seconds
        Window-by-window intersection  = 0.0314254 seconds per thread
        Breakpoint search              = 0.00556094 seconds per thread
        Recording result               = 0.0035269 seconds per thread
    Imputation                      = 9.05471 seconds

 22.651812 seconds (78.25 M allocations: 6.328 GiB, 4.97% gc time)
error_rate = 9.775004726791454e-5


# Rescreen

In [4]:
# keep best pair only (8 thread)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, rescreen=true);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Phasing chunk 1/1...100%|███████████████████████████████| Time: 0:00:14
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.07889 seconds
    Computing haplotype pair        = 14.6762 seconds
        BLAS3 mul! to get M and N      = 0.0546491 seconds per thread
        haplopair search               = 1.54516 seconds per thread
        min least sq on observed data  = 0.0497179 seconds per thread
        finding redundant happairs     = 0.0706418 seconds per thread
    Phasing by win-win intersection = 0.289549 seconds
    Imputation                      = 6.7911 seconds

 29.900606 seconds (77.39 M allocations: 7.481 GiB, 4.44% gc time)
error_rate = 8.100052272752551e-5


# Screen for flanking windows

In [3]:
# keep best pair only (8 thread)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.69623 seconds
    Computing haplotype pair        = 3.37601 seconds
        BLAS3 mul! to get M and N      = 0.0963943 seconds per thread
        haplopair search               = 2.89449 seconds per thread
        finding redundant happairs     = 0.0482441 seconds per thread
    Phasing by win-win intersection = 0.447916 seconds
    Imputation                      = 6.62697 seconds

 19.876269 seconds (77.16 M allocations: 7.417 GiB, 5.39% gc time)
error_rate = 7.657402155416903e-5


# Haplotype thinning

In [ ]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=100, max_haplotypes=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06


In [5]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=100, max_haplotypes=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.57389 seconds
    Computing haplotype pair        = 3.81197 seconds
        screening for top haplotypes   = 0.738086 seconds per thread
        BLAS3 mul! to get M and N      = 2.67268 seconds per thread
        haplopair search               = 0.164605 seconds per thread
        initializing missing           = 0.166272 seconds per thread
        allocating and viewing         = 0.0125657 seconds per thread
        index conversion               = 0.00055434 seconds per thread
    Phasing by win-win intersection = 0.0602829 seconds
        Window-by-window intersection  = 0.0470093 seconds per thread
        Breakpoint search              = 0.00641138 seconds per thread
        Recording result               = 0.00341723 seconds per thread
    Imputation                      = 7.06285 seconds

 19.509919 seconds (76.67 M allocations: 6.657 GiB, 5.16% gc time)
error_rat

# Try stepwise search heuristic

In [6]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, max_haplotypes=100, stepwise=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 7.93711 seconds
    Computing haplotype pair        = 1.14714 seconds
        BLAS3 mul! to get M and N      = 0.073336 seconds per thread
        haplopair search               = 0.881813 seconds per thread
        initializing missing           = 0.126261 seconds per thread
        allocating and viewing         = 0.0130243 seconds per thread
        index conversion               = 0.000442667 seconds per thread
    Phasing by win-win intersection = 0.051122 seconds
        Window-by-window intersection  = 0.0405628 seconds per thread
        Breakpoint search              = 0.00488876 seconds per thread
        Recording result               = 0.00270064 seconds per thread
    Imputation                      = 6.45776 seconds

 15.594449 seconds (76.67 M allocations: 6.378 GiB, 5.74% gc time)
error_rate = 8.563833928352964e-5


# Beagle 5.1

In [ ]:
# convert to bref3 (run in terminal)
java -jar ../bref3.18May20.d20.jar ref.excludeTarget.vcf.gz > ref.excludeTarget.bref3 

In [31]:
# run beagle 5 (8 thread)
run(`java -jar beagle.18May20.d20.jar gt=compare2/target.typedOnly.maf0.01.masked.vcf.gz ref=compare2/ref.excludeTarget.bref3 out=compare2/beagle.result nthreads=8`)

# beagle 5 error rate
X_complete = convert_gt(Float32, "compare2/target.full.vcf.gz")
n, p = size(X_complete)
X_beagle = convert_gt(Float32, "compare2/beagle.result.vcf.gz")
error_rate = sum(X_beagle .!= X_complete) / n / p

beagle.18May20.d20.jar (version 5.1)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.18May20.d20.jar" to list command line argument
Start time: 01:01 AM PDT on 30 Jun 2020

Command line: java -Xmx3641m -jar beagle.18May20.d20.jar
  gt=compare2/target.typedOnly.maf0.01.masked.vcf.gz
  ref=compare2/ref.excludeTarget.bref3
  out=compare2/beagle.result
  nthreads=8

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:      19,000
Study samples:           1,000

Window 1 (1:34-9999816)
Reference markers:      89,913
Study markers:          36,874

Burnin  iteration 1:           36 seconds
Burnin  iteration 2:           32 seconds
Burnin  iteration 3:           35 seconds
Burnin  iteration 4:           43 seconds
Burnin  iteration 5:           37 seconds
Burnin  iteration 6:           1 minute 16 seconds

Phasing iteration 1:           2 minutes 0 seconds
Phasing iteration 2:           41 seconds
Phasing iteration 3:           38 seconds
Phasing iteration 4:  

1.7794979591382782e-5

# Minimac4 error

Need to first convert reference vcf file to m3vcf using minimac3 (on Hoffman)

```Julia
minimac3 = "/u/home/b/biona001/haplotype_comparisons/minimac3/Minimac3/bin/Minimac3"
@time run(`$minimac3 --refHaps haplo_ref.vcf.gz --processReference --prefix haplo_ref`)
```

In [ ]:
# use eagle 2.4 for prephasing

In [ ]:
# run minimac 4
minimac4 = "/Users/biona001/Benjamin_Folder/UCLA/research/softwares/Minimac4/build/minimac4"
run(`$minimac4 --refHaps haplo_ref.m3vcf.gz --haps target_masked.vcf.gz --prefix minimac4.result`)
    
X_minimac = convert_gt(Float32, "minimac4.result.dose.vcf.gz", as_minorallele=false)
error_rate = sum(X_minimac .!= X_complete) / n / p

# BLAS 3

In [2]:
# account for allele frequency (8 thread, BLAS3)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=2000, thinning_scale_allelefreq=false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:19
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 11.2299 seconds
    Computing haplotype pair        = 20.639 seconds
        computing dist(X, H)           = 0.10026 seconds per thread
        BLAS3 mul! to get M and N      = 0.0936877 seconds per thread
        haplopair search               = 13.9145 seconds per thread
        finding redundant happairs     = 0.0360827 seconds per thread
    Phasing by win-win intersection = 1.08454 seconds
    Imputation                      = 8.11408 seconds

 59.805070 seconds (161.01 M allocations: 12.053 GiB, 4.43% gc time)
error_rate = 8.47485903039605e-5


# BLAS 2

In [ ]:
# account for allele frequency (8 thread, BLAS3)
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=2000, thinning_scale_allelefreq=false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Computing optimal haplotype pairs... 71%|██████████▋    |  ETA: 0:01:21